In [0]:
dbutils.fs.ls('/databricks-datasets/nyctaxi/tables/nyctaxi_yellow/')

In [0]:
filepath = '/databricks-datasets/nyctaxi/readme_nyctaxi.txt'

with open(filepath, 'r') as f:
  print(f.read())

In [0]:
df = spark.read.format('delta').load('dbfs:/databricks-datasets/nyctaxi/tables/nyctaxi_yellow/')
df.display()

In [0]:
df.count()

In [0]:
from pyspark.sql.functions import col, round
df = df.withColumn('fare_amount_gbp', round(col('fare_amount') / 1.35, 2))
df.display()


In [0]:
df_single = df.where('passenger_count = 1')
df_multiple = df.where('passenger_count > 1')
print(f'There are {df_single.count()} single passenger journeys and {df_multiple.count()} multiple passenger journeys in the datasets. The total number of journeys is {df.count()}')

In [0]:
def format_as_currency(value):
    return "${:,.2f}".format(value)

In [0]:
total_fare_amount = df.select('fare_amount').agg({'fare_amount': 'sum'}).collect()[0]['sum(fare_amount)']
avg_fare_amount = df.select('fare_amount').agg({'fare_amount': 'avg'}).collect()[0]['avg(fare_amount)']
print(f'The total fare amount for all journeys is {format_as_currency(total_fare_amount)}. Average fare is: {format_as_currency(avg_fare_amount)}')

In [0]:

df.createOrReplaceTempView('tmp_nyctaxi')

sql_agg = f'''
select sum(fare_amount) as total_fare_amount, avg(fare_amount) as avg_fare_amount
from tmp_nyctaxi
'''
df_aggs = spark.sql(sql_agg)
total_fare_amount_sql = df_aggs.select('total_fare_amount').collect()[0]['total_fare_amount']
avg_fare_amount_sql = df_aggs.select('avg_fare_amount').collect()[0]['avg_fare_amount']
print(f'The total fare amount for all journeys is {format_as_currency(total_fare_amount_sql)}. Average fare is: {format_as_currency(avg_fare_amount_sql)}')
